# The Palatable Diet Problem  

**Problem Description.** The Diet Problem is the first large-scale optimization problem to be solved with the Simplex algorithm by Jack Laderman in [1947](https://www.mpi-inf.mpg.de/fileadmin/inf/d1/teaching/winter18/Ideen/Materialien/Dantzig-Diet.pdf). The basic formulation of this problem consists of minimizing the cost of a food basket while meeting the specified nutrient requirements. In this notebook, we solve The Palatable Diet Problem (TPDP), where the basic model is extended with a constraint on the food basket palatability. An explicit formula of the palatability constraint is unknown, but we have data on several food baskets and the respective palatability score. First, we define a conceptual model with the *known constraint*. Then, OptiCL is used to learn and embed the palatability constraint.  
(*TPDP is part of a larger optimization problem which simultaneously  optimizes  the  food  basket  to  be  delivered,  the  sourcing  plan,  the  delivery  plan, and  the  transfer  modality  of  a  month-long  food  supply in a Wolrd Food Program setting ([Maragno et al., 2021])*).

**Objective function:** minimize the total cost of the food basket.  
$\min_{\boldsymbol{x}} c^\top \boldsymbol{x}$

*subject to* 

**Nutritional constraints:** for each nutrient $j\in\mathcal{N}$, at least meet the minimum required level.  
$ \sum_{k \in \mathcal{K}} nutval_{kj} x_{k} \geq nutreq_{j}, \ \ \ \forall l\in\mathcal{N},$   
**Constraints on sugar and salt.**</font>  
$ x_{salt} = 5,$   
$ x_{sugar} = 20,$  
**Palatability constraints:** the food basket palatability has to be at least equal to $t$.  
$ y \geq t,$  
**Learned predictive model:** the palatability is defined using a predictive model.  
$ y = \hat{h}(\boldsymbol{x}),$   
**Non negativity constraints.**  
$ x_{k} \geq 0, \ \ \ \forall k \in \mathcal{K}.$  

In [1]:
import pandas as pd
from imp import reload
import numpy as np
import math
from sklearn.utils.extmath import cartesian
import time
import sys
import os
sys.path.append(os.path.abspath('../../src'))
import opticl
from pyomo import environ
from pyomo.environ import *
np.random.seed(0)

### Data Loading  
**nutr_val**: nutritional values for each of the 25 foods  
**nutr_req**: 11 nutrition requirements  
**cost_p**: vector of procurement costs  
**dataset**: dataframe of food basket instances and relative palatability score

In [2]:
nutr_val = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_val', index_col='Food', engine='openpyxl')
nutr_req = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_req', index_col='Type', engine='openpyxl')
cost_p = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='FoodCost', index_col='Supplier', engine='openpyxl').iloc[0,:]
dataset = pd.read_csv('processed-data/WFP_dataset.csv').sample(frac=1)
dataset

,Beans,Bulgur,Cheese,Fish,Meat,CSB,Dates,DSM,Milk,Salt,...,Soya-fortified bulgur wheat,Soya-fortified maize meal,Soya-fortified sorghum grits,Soya-fortified wheat flour,Sugar,Oil,Wheat,Wheat flour,WSB,label
398,0.687675,1.257354,0.000000,0.0,0.0,0.000000,0.000000,0.302104,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.357429,2.823603,0.000000,0.637964,0.715428
3833,0.551125,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.117990,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.392274,2.540599,3.414615,0.733300,0.292719
4836,0.701614,0.000000,0.000000,0.0,0.0,0.094990,0.000000,0.330808,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.221908,0.336647,0.000000,0.545864,0.816616
4572,0.000000,3.832166,0.000000,0.0,0.0,0.000000,0.626751,0.278648,0.132718,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.311117,0.000000,0.000000,0.694007,0.794680
636,0.039754,0.000000,0.344293,0.0,0.0,0.000000,0.000000,0.106482,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.160220,0.000000,0.000000,0.788790,0.261417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,0.743757,0.000000,0.115426,0.0,0.0,0.700000,0.000000,0.479955,0.000000,0.05,...,0.0,0.041521,0.0,0.0,0.2,0.181913,0.000000,0.000000,0.000000,0.210851
3264,0.779749,0.000000,0.075277,0.0,0.0,0.761232,0.000000,0.392905,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.178567,0.000000,0.000000,0.029013,0.293570
1653,0.668713,3.322546,0.000000,0.0,0.0,0.429041,0.000000,0.450216,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.271382,0.000000,0.000000,0.189249,0.813417
2607,0.460621,1.464405,0.160834,0.0,0.0,0.757835,0.000000,0.391194,0.000000,0.05,...,0.0,0.000000,0.0,0.0,0.2,0.166244,0.000000,0.000000,0.000000,0.382687


# OptiCL: Optimization with Constraint Learning

## Step 1: Conceptual Model

In [3]:
def init_conceptual_model(cost_p):
    N = list(nutr_val.index)  # foods
    M = nutr_req.columns  # nutrient requirements

    model = ConcreteModel('TPDP')

    '''
    Decision variables
    '''
    model.x = Var(N, domain=NonNegativeReals)  # variables controlling the food basket

    '''
    Objective function.
    '''
    def obj_function(model):
        return sum(cost_p[food].item()*model.x[food] for food in N)

    model.OBJ = Objective(rule=obj_function, sense=minimize)

    '''
    Nutrients requirements constraint.
    '''
    def constraint_rule1(model, req):
        return sum(model.x[food] * nutr_val.loc[food, req] for food in N) >= nutr_req[req].item()
    model.Constraint1 = Constraint(M, rule=constraint_rule1)
    '''
    Sugar constraint
    '''
    def constraint_rule2(model):
        return model.x['Sugar'] == 0.2
    model.Constraint2 = Constraint(rule=constraint_rule2)
    '''
    Salt constraint
    '''
    def constraint_rule3(model):
        return model.x['Salt'] == 0.05
    model.Constraint3 = Constraint(rule=constraint_rule3)
    
    return model

## Step 2: Data Processing

In [4]:
y = dataset['label']
X = dataset.drop(['label'], axis=1, inplace=False)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

## Step 3: Learn the predictive models
'alg_list' specifies the list of algorithms that you will consider in the training pipeline. If you have the InterpretableAI license, you can include **iai** (Optimal Trees with Hyperplanes) or **iai-single** (Optimal Trees with single feature splits) in the list. If using IAI, you must specify the metric as 'r2'. Otherwise, the default metric is 'neg_squared_mse'.

In [15]:
version = 'TPDP_v1'
alg_list = ['mlp', 'linear', 'gbm', 'cart', 'rf', 'svm']
outcome_list = {'palatability_1': {'outcome_type': ['constraint', None], 'task_type': 'continuous', 'alg_list':['mlp', 'linear'], 
                                   'X_train':X_train, 'y_train':y_train, 'X_test':X_test, 'y_test':y_test}, 
               'palatability_2': {'outcome_type': ['objective', 1], 'task_type': 'continuous', 'alg_list':['gbm', 'cart'],
                                 'X_train':X_train, 'y_train':y_train, 'X_test':X_test, 'y_test':y_test}}  # Constraint to be learned

#### Train models (or skip if pre-saved)  
The training will use only regression models. 

In [16]:
performance = pd.DataFrame()

if not os.path.exists('results/'):
    os.makedirs('results/')

for outcome in outcome_list.keys():
    print(f'Learning a constraint for {outcome}')
    
    alg_list = outcome_list[outcome]['alg_list']
    task_type = outcome_list[outcome]['task_type']
    for alg in alg_list:
        X_train = outcome_list[outcome]['X_train']
        y_train = outcome_list[outcome]['y_train']
        X_test = outcome_list[outcome]['X_test']
        y_test = outcome_list[outcome]['y_test']
        
        if not os.path.exists('results/%s/' % alg):
            os.makedirs('results/%s/' % alg)
        print(f'Training {alg}')
        s = 1

        ## Run shallow/small version of RF
        alg_run = 'rf_shallow' if alg == 'rf' else alg

        m, perf = opticl.run_model(X_train, y_train, X_test, y_test, alg_run, outcome, task = task_type,
                               seed = s, cv_folds = 5, 
                               # metric = 'r2',
                               save = False
                              )

        ## Save model
        constraintL = opticl.ConstraintLearning(X_train, y_train, m, alg)
        constraint_add = constraintL.constraint_extrapolation(task_type)
        constraint_add.to_csv('results/%s/%s_%s_model.csv' % (alg, version, outcome), index = False)

        ## Extract performance metrics
        try:
            perf['auc_train'] = roc_auc_score(y_train >= threshold, m.predict(X_train))
            perf['auc_test'] = roc_auc_score(y_test >= threshold, m.predict(X_test))
        except: 
            perf['auc_train'] = np.nan
            perf['auc_test'] = np.nan

        perf['seed'] = s
        perf['outcome'] = outcome
        perf['alg'] = alg
        perf['save_path'] = 'results/%s/%s_%s_model.csv' % (alg, version, outcome)
        
            
        perf.to_csv('results/%s/%s_%s_performance.csv' % (alg, version, outcome), index = False)
        
        performance = performance.append(perf)
        print()
print('Saving the performance...')
performance.to_csv('results/%s_performance.csv' % version, index = False)
print('Done!')

Learning a constraint for palatability_1
Training mlp
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = mlp, metric = None
saving... results/mlp_palatability_1_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.011064680927472396
Train R2: 0.7832510216353195
-------------------testing evaluation-----------------------
Test MSE: 0.018394522665774987
Test R2: 0.6418989069753125

Training linear
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = linear, metric = None
saving... results/linear_palatability_1_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.04494230882369555
Train R2: 0.11961315588417554
-------------------testing evaluation-----------------------
Test MSE: 0.04647042488484837
Test R2: 0.095322545

C:\Users\dmaragn\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
C:\Users\dmaragn\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
C:\Users\dmaragn\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
C:\Users\dmaragn\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
C:\Users\dmaragn\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
C:\Users\dmaragn\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeRegr


Training cart
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = cart, metric = None
saving... results/cart_palatability_2_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.010676479379551226
Train R2: 0.7908556050356943
-------------------testing evaluation-----------------------
Test MSE: 0.015873275949482357
Test R2: 0.6909820618521263

Saving the performance...
Done!


In [7]:
performance

,save_path,seed,cv_folds,task,parameters,best_params,valid_score,train_score,train_r2,test_score,test_r2,auc_train,auc_test,outcome,alg
0,results/mlp/TPDP_v1_palatability_1_model.csv,1,5,continuous,"{'hidden_layer_sizes': [(10,), (20,), (50,), (...","{'hidden_layer_sizes': (100,)}",-0.019855,0.011065,0.783251,0.018395,0.641899,NaN,NaN,palatability_1,mlp
0,results/linear/TPDP_v1_palatability_1_model.csv,1,5,continuous,"{'alpha': [0.1, 1, 10, 100, 1000], 'l1_ratio':...","{'alpha': 0.1, 'l1_ratio': 0.1}",-0.045594,0.044942,0.119613,0.046470,0.095323,NaN,NaN,palatability_1,linear
0,results/gbm/TPDP_v1_palatability_2_model.csv,1,5,continuous,"{'learning_rate': [0.01, 0.025, 0.05, 0.075, 0...","{'learning_rate': 0.2, 'max_depth': 5, 'n_esti...",-0.008343,0.003146,0.938371,0.008664,0.831326,NaN,NaN,palatability_2,gbm
0,results/cart/TPDP_v1_palatability_2_model.csv,1,5,continuous,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 10, 'max_features': 1.0, 'min_sa...",-0.016014,0.010676,0.790856,0.015873,0.690982,NaN,NaN,palatability_2,cart


## Step 4: Predictive model selection and Optimization

In [8]:
performance = pd.read_csv('results/%s_performance.csv' % version)
performance.dropna(axis='columns')

,save_path,seed,cv_folds,task,parameters,best_params,valid_score,train_score,train_r2,test_score,test_r2,outcome,alg
0,results/mlp/TPDP_v1_palatability_1_model.csv,1,5,continuous,"{'hidden_layer_sizes': [(10,), (20,), (50,), (...","{'hidden_layer_sizes': (100,)}",-0.019855,0.011065,0.783251,0.018395,0.641899,palatability_1,mlp
1,results/linear/TPDP_v1_palatability_1_model.csv,1,5,continuous,"{'alpha': [0.1, 1, 10, 100, 1000], 'l1_ratio':...","{'alpha': 0.1, 'l1_ratio': 0.1}",-0.045594,0.044942,0.119613,0.046470,0.095323,palatability_1,linear
2,results/gbm/TPDP_v1_palatability_2_model.csv,1,5,continuous,"{'learning_rate': [0.01, 0.025, 0.05, 0.075, 0...","{'learning_rate': 0.2, 'max_depth': 5, 'n_esti...",-0.008343,0.003146,0.938371,0.008664,0.831326,palatability_2,gbm
3,results/cart/TPDP_v1_palatability_2_model.csv,1,5,continuous,"{'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_...","{'max_depth': 10, 'max_features': 1.0, 'min_sa...",-0.016014,0.010676,0.790856,0.015873,0.690982,palatability_2,cart


In [10]:
model_master = opticl.model_selection(performance, outcome_list)
model_master[['lb', 'ub', 'SCM_counterfactuals', 'features', 'trust_region', 'dataset_path', 'clustering_model', 'max_violation']] = None

model_master.loc[0, 'lb'] = 0.5
model_master.loc[0, 'ub'] = None
model_master.loc[0, 'SCM_counterfactuals'] = None
model_master.at[0, 'features'] = [col for col in X.columns]
model_master.loc[0, 'trust_region'] = True
model_master.loc[0, 'dataset_path'] = 'processed-data/WFP_dataset.csv'
model_master.loc[0, 'clustering_model'] = None
model_master.loc[0, 'max_violation'] = None

model_master.loc[1, 'lb'] = None
model_master.loc[1, 'ub'] = None
model_master.loc[1, 'SCM_counterfactuals'] = None
model_master.at[1, 'features'] = [col for col in X.columns]
model_master.loc[1, 'trust_region'] = False
model_master.loc[1, 'dataset_path'] = 'processed-data/WFP_dataset.csv'
model_master.loc[1, 'clustering_model'] = None
model_master.loc[1, 'max_violation'] = None


model_master

          outcome model_type                                     save_path  \
0  palatability_1        mlp  results/mlp/TPDP_v1_palatability_1_model.csv   
1  palatability_2        gbm  results/gbm/TPDP_v1_palatability_2_model.csv   

         task  objective  
0  continuous          0  
1  continuous          1  


,outcome,model_type,save_path,task,objective,lb,ub,SCM_counterfactuals,features,trust_region,dataset_path,clustering_model,max_violation
0,palatability_1,mlp,results/mlp/TPDP_v1_palatability_1_model.csv,continuous,0,0.5,None,None,"[Beans, Bulgur, Cheese, Fish, Meat, CSB, Dates...",True,processed-data/WFP_dataset.csv,None,None
1,palatability_2,gbm,results/gbm/TPDP_v1_palatability_2_model.csv,continuous,1,None,None,None,"[Beans, Bulgur, Cheese, Fish, Meat, CSB, Dates...",False,processed-data/WFP_dataset.csv,None,None


In [11]:
def getSolution(model, X):
    solution = {}
    palatability = 0
    count = 0
    for v in model.getVars():
        if 'x[' in v.varName:
            solution[list(X.columns)[count]]=[v.x]
            print(v.varName)
            count += 1
    return solution

In [12]:
result = {}
conceptual_model= init_conceptual_model(cost_p)
MIP_final_model = opticl.optimization_MIP(conceptual_model, model_master)
opt = SolverFactory('gurobi')
print('---------------------Solving the optimization problem---------------------')
results = opt.solve(MIP_final_model) 
solution = {}
for food in  list(nutr_val.index):
    if value(MIP_final_model.x[food])*100 > 0.0000001:
        solution[food] = str(np.round(value(MIP_final_model.x[food])*100, 2))+'g'
print('The optimal solution is: \n', solution)
print(f"The predicted palatability of the optimal solution is {value(MIP_final_model.y['palatability_1'])}")

Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability_1
['Beans', 'Bulgur', 'Cheese', 'Fish', 'Meat', 'CSB', 'Dates', 'DSM', 'Milk', 'Salt', 'Lentils', 'Maize', 'Maize meal', 'Chickpeas', 'Rice', 'Sorghum', 'Soya-fortified bulgur wheat', 'Soya-fortified maize meal', 'Soya-fortified sorghum grits', 'Soya-fortified wheat flour', 'Sugar', 'Oil', 'Wheat', 'Wheat flour', 'WSB']
Embedding objective function for palatability_2
---------------------Solving the optimization problem---------------------
The optimal solution is: 
 {'Beans': '16.05g', 'DSM': '0.02g', 'Milk': '56.78g', 'Salt': '5.0g', 'Lentils': '35.97g', 'Maize': '130.97g', 'Chickpeas': '0.33g', 'Sugar': '20.0g', 'Oil': '24.04g', 'Wheat': '215.21g', 'WSB': '70.0g'}
The predicted palatability of the optimal solution is 0.6518423667038796
